In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import operator
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom

from pymedphys._experimental.fileformats import mephysto as pp_mephysto
from pymedphys import dicom as pp_dicom

In [ ]:
ROOT_DIR = Path(r"S:\Physics\Monaco\Model vs Measurement Comparisons")

DICOM_DIR = ROOT_DIR.joinpath(r"Beam Models\CCA Monaco Photon Monte Carlo\\DICOM dose exports")
MEASUREMENTS_DIR = ROOT_DIR.joinpath(r"Measurements\RCCC\Photons")
RESULTS = ROOT_DIR.joinpath(r"Results\RCCC\pymedphys-based-results")

In [ ]:
absolute_dose_table = pd.read_csv(MEASUREMENTS_DIR.joinpath('AbsoluteDose.csv'), index_col=0)
absolute_dose = absolute_dose_table['d10 @ 90 SSD']['06MV']
absolute_dose

In [ ]:
output_factors = pd.read_csv(MEASUREMENTS_DIR.joinpath('OutputFactors.csv'), index_col=0)
output_factors

In [ ]:
absolute_doses = {
    key: output_factors[key]['06MV'] * absolute_dose
    for key in output_factors.columns
}

absolute_doses

In [ ]:
pp_mephysto.load_mephysto_directory?

In [ ]:
absolute_scans_per_field = pp_mephysto.load_mephysto_directory(
    MEASUREMENTS_DIR, r'06MV (\d\dx\d\d Open)\.mcc', absolute_doses, 100)

getter = operator.itemgetter('displacement', 'dose')

In [ ]:
keys = absolute_scans_per_field.keys()

In [ ]:
def load_dicom_files(directory, keys):
    dicom_file_map = {
        key: directory.joinpath(f'06MV {key}.dcm')
        for key in keys
    }
    
    dicom_dataset_map = {
        key: pydicom.read_file(str(dicom_file_map[key]), force=True)
        for key in keys
    }
    
    return dicom_dataset_map

In [ ]:
dicom_dataset_map = load_dicom_files(DICOM_DIR, keys)
dicom_plan = pydicom.read_file(DICOM_DIR.parent.joinpath("plan.dcm"), force=True)

In [ ]:
def plot_one_axis(ax, displacement, meas_dose, tps_dose):
    diff = tps_dose - meas_dose
    
    lines = []
    
    lines += ax.plot(displacement, meas_dose, label='Measured Dose')
    lines += ax.plot(displacement, tps_dose, label='TPS Dose')
    ax.set_ylabel('Dose (Gy / 100 MU)')
    
    x_bounds = [np.min(displacement), np.max(displacement)]
    ax.set_xlim(x_bounds)

    ax_twin = ax.twinx()

    lines += ax_twin.plot(displacement, diff, color='C3', alpha=0.5, label='Residuals [TPS - Meas]')
    ax_twin.plot(x_bounds, [0, 0], '--', color='C3', lw=0.5)
    ax_twin.set_ylabel('Dose difference [TPS - Meas] (Gy / 100 MU)')

    labels = [l.get_label() for l in lines]
    
    ax.legend(lines, labels)
    
    return ax_twin



def plot_tps_meas_diff(displacement, meas_dose, tps_dose):
    fig, ax = plt.subplots(1, 1, figsize=(12,6))
    
    plot_one_axis(ax, displacement, meas_dose, tps_dose)
    
    return fig, ax


def plot_pdd_diff(key):
    depth, meas_dose = getter(absolute_scans_per_field[key]['depth_dose'])
    tps_dose = pp_dicom.depth_dose(depth, dicom_dataset_map[key], dicom_plan) / 10

    fig, ax = plot_tps_meas_diff(depth, meas_dose, tps_dose)
    ax.set_title(f'Depth Dose | {key} field', fontsize="x-large")
    

for key in keys:
    plot_pdd_diff(key)
    filename = RESULTS.joinpath(f'{key}_pdd.png')
    plt.savefig(filename)
    plt.show()

In [ ]:
def plot_profile_diff(key, depth, direction):
    displacement, meas_dose = getter(absolute_scans_per_field[key]['profiles'][depth][direction])
    
    tps_dose = pp_dicom.profile(displacement, depth, direction, dicom_dataset_map[key], dicom_plan) / 10

    fig, ax = plot_tps_meas_diff(displacement, meas_dose, tps_dose)
    ax.set_title(f'{direction.capitalize()} Profile | {key} field | Depth: {depth} mm', fontsize="x-large")
    

for key in keys:
    depths = absolute_scans_per_field[key]['profiles'].keys()
    for depth in depths:
        for direction in ['inplane', 'crossplane']:
            plot_profile_diff(key, depth, direction)
            filename = RESULTS.joinpath(f'{key}_profile_{depth}mm_{direction}.png')
            plt.savefig(filename)
            plt.show()